In [ ]:
# 旋转矫正展示

from imutils.perspective import four_point_transform
import numpy as np
import argparse
import imutils
import cv2
import math
from numpy import average, dot, linalg
import time
from PIL import Image, ImageDraw, ImageFont
import os




def template_demo(tpl):
    target = cv2.imread("./res_gray.png")
    th, tw = tpl.shape[:2]
    result = cv2.matchTemplate(target, tpl, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    tl=max_loc
    br = (tl[0] + tw, tl[1] + th)  # br是矩形右下角的点的坐标
    point=(int((tl[0]+br[0])/2),int((tl[1]+br[1])/2))#计算矩形中心
    cv2.rectangle(target, tl, br, (0, 0, 255), 2)
    cv2.circle(target,point,3,(0,255,0))
#     print(point)
    index=int(point[1]/100)*7+int(point[0]/100)
#     print(index)
    cv2.namedWindow("match")
    cv2.imshow("match", target)
    return index

def add_chinese(img,name,text_position):

    # 图像从OpenCV格式转换成PIL格式
    img_PIL = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
 
    # 字体  
    font = ImageFont.truetype('simhei.ttf', 20, encoding="utf-8")
    
    # 开始显示
    draw = ImageDraw.Draw(img_PIL)
    draw.text(text_position, '识别结果：' + name, font=font, fill=(0,0,255))

    # 转换回OpenCV格式
    img_OpenCV = cv2.cvtColor(np.asarray(img_PIL),cv2.COLOR_RGB2BGR)

    return img_OpenCV




kapian = ['(', ')', '+', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'A', 'B', 'C', '*', 
          '/', 'D', '掉头', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', '前进', 
          'T', '停车', 'U', 'V', 'W', 'X', 'Y', '右转', 'Z', '左转']
sxt = 0
capture = cv2.VideoCapture(sxt)
while True:
    ret, frame = capture.read()
    if sxt:
        frame = cv2.flip(frame,1)
    
    if ret is True:
        start = time.time()
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
#         warped = four_point_transform(gray, docCnt.reshape(4, 2))
        # 对灰度图应用大津二值化算法
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        # 在二值图像中查找轮廓，然后初始化题目对应的轮廓列表
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edged = cv2.Canny(blurred, 75, 200)
        
        cnts,_ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#         cnts = cnts[1] if imutils.is_cv3() else cnts[0]
        questionCnts = []
#         print(len(cnts))
        # 对每一个轮廓进行循环处理
        for c in cnts:
            # 计算轮廓的边界框，然后利用边界框数据计算宽高比
            (x, y, w, h) = cv2.boundingRect(c)
            ar = w / float(h)
            # 构造只有当前气泡轮廓区域的掩模图像
            mask = np.zeros(thresh.shape, dtype="uint8")
            cv2.drawContours(mask, [c], -1, 255, -1)

            # 对二值图像应用掩模图像，然后就可以计算气泡区域内的非零像素点。
            mask = cv2.bitwise_and(thresh, thresh, mask=mask)
            total = cv2.countNonZero(mask)
#             print(total)
            ttt = np.mean(mask)
#             print(ttt)
    
    
    
            # 大小、像素点、比例、参数设置处
            if w <= 60 and h <= 60 and w >= 10 and h >= 10 and total > 150 and ar >= 0.9 and ar <= 1.1:
                tmp_yx = np.array([int(x+w/2),int(y+h/2),int(w/2)])
                questionCnts.append(tmp_yx)
#                 cv2.circle(frame,(int(x+w/2),int(y+h/2)),int(w/2),(255,0,0),2)
                
                
                
                
        # 对所有点进行判断，构建出三点为一个集合体。
#         print(questionCnts)
        tmp_jh = []
        arr_jh = []
        all_jh = []
#         print(len(questionCnts))
        for i in range(len(questionCnts)):
            if (i not in tmp_jh):# and (i not in all_jh):
                tmp_jh.append(i)
                all_jh.append(i)
            
            low_lenth = 10 * questionCnts[i][2]
            upper_lenth = 14 * questionCnts[i][2]
            for j in range(len(questionCnts)):
                if (math.sqrt(math.pow(questionCnts[i][0] - questionCnts[j][0], 2) + 
                              math.pow(questionCnts[i][1] - questionCnts[j][1], 2)) > low_lenth) & \
                (math.sqrt(math.pow(questionCnts[i][0] - questionCnts[j][0], 2) + 
                           math.pow(questionCnts[i][1] - questionCnts[j][1], 2)) < upper_lenth):
                    if (j not in tmp_jh):# and (j not in all_jh):
                        tmp_jh.append(j)
                        all_jh.append(j)
            if len(tmp_jh) == 3:
                arr_jh.append(tmp_jh)
                tmp_jh = []
#         print(arr_jh)
        
        # 计算第四个点
        for i in range(len(arr_jh)):
#             print(len(arr_jh))
            for j in range(3):
                tmp_point1 = questionCnts[arr_jh[i][(0+j)%3]]
                tmp_point2 = questionCnts[arr_jh[i][(1+j)%3]]
                tmp_point3 = questionCnts[arr_jh[i][(2+j)%3]]



                if ((math.sqrt(math.pow(tmp_point1[0] - tmp_point2[0], 2) + math.pow(tmp_point1[1] - tmp_point2[1], 2)) / \
                math.sqrt(math.pow(tmp_point1[0] - tmp_point3[0], 2) + math.pow(tmp_point1[1] - tmp_point3[1], 2))) > 0.8) & \
                ((math.sqrt(math.pow(tmp_point1[0] - tmp_point2[0], 2) + math.pow(tmp_point1[1] - tmp_point2[1], 2)) / \
                math.sqrt(math.pow(tmp_point1[0] - tmp_point3[0], 2) + math.pow(tmp_point1[1] - tmp_point3[1], 2))) < 1.2):
                    point1 = tmp_point1
                    point2 = tmp_point2
                    point3 = tmp_point3
#                     print('nihaonihao')
                    break
            try:
                point4 = (point2[0]+point3[0]-point1[0],point2[1]+point3[1]-point1[1])
                cv2.circle(frame, (point1[0], point1[1]), 1, (0, 0, 255), 15)
#                 print('测试点1')
                if math.sqrt(math.pow(point1[0] - point2[0], 2) + math.pow(point1[1] - point2[1], 2)) > \
                math.sqrt(math.pow(point1[0] - point4[0], 2) + math.pow(point1[1] - point4[1], 2)):
#                     print('测试点2')
                    pass
                else:
#                     print('测试点3')
                    
                    cv2.line(frame, (point1[0], point1[1]), (point2[0], point2[1]), (0, 255, 0), 1, 4)
                    cv2.line(frame, (point2[0], point2[1]), (point4[0], point4[1]), (0, 255, 0), 1, 4)
                    cv2.line(frame, (point3[0], point3[1]), (point4[0], point4[1]), (0, 255, 0), 1, 4)
                    cv2.line(frame, (point3[0], point3[1]), (point1[0], point1[1]), (0, 255, 0), 1, 4)
                    
                    # 图像校正部分
                    # 根据点一和其他两个点的位置，旋转图片。
#                     print('测试点6')
                    pts = np.array([(point1[0], point1[1]), (point2[0], point2[1]), (point3[0], point3[1]), (point4[0], point4[1])])
#                     print(pts)
#                     print('测试点7')
                    warped = four_point_transform(frame, pts)
    
#                     print('测试点8')
                    zuoshang = warped[2:8, 2:8]
                    youshang = warped[2:8, -8:-2]
                    zuoxia = warped[-8:-2, 2:8]
                    youxia = warped[-8:-2, -8:-2]
#                     print(np.mean(zuoshang))
#                     print(np.mean(youshang))
#                     print(np.mean(zuoxia))
#                     print(np.mean(youxia))
                    tmp_war = warped
                    if np.mean(zuoshang) > 100:
                        tmp_war=np.rot90(warped)
                        tmp_war=np.rot90(tmp_war)
#                         print('111111111111111111111')
                    if np.mean(youshang) > 100:
                        tmp_war=np.rot90(warped)
                        tmp_war=np.rot90(tmp_war)
                        tmp_war=np.rot90(tmp_war)
#                         print('2222222222222222222222222')
                    if np.mean(zuoxia) > 100:
                        tmp_war=np.rot90(warped)
#                         print('3333333333333333333333333333333')
                    if np.mean(youxia) > 100:
                        tmp_war=warped
#                         print('444444444444444444444')
#                     if tmp_war.shape[0] > 140:
                    clip_jc = tmp_war[int(tmp_war.shape[0]/5):-int(tmp_war.shape[0]/5), int(tmp_war.shape[0]/5):-int(tmp_war.shape[0]/5)]
#                     print(tmp_war.shape)
#                     else:
#                         clip_jc = tmp_war[20:-20, 20:-20]
                    cv2.imshow('warped', clip_jc)
                    
                    # 模板匹配部分
                    kapian_name = kapian[template_demo(clip_jc)]
#                     frame = add_chinese(frame,kapian_name,(int(point4[0]), int(point4[1])-30))
                    frame = add_chinese(frame,kapian_name,(0, 0))
                    
            except:# Exception as e:
                pass
#                 print('str(e):\t\t', str(e))
#                 print('repr(e):\t', repr(e))
#                 print('traceback.print_exc():', traceback.print_exc())
#                 print('traceback.format_exc():\n%s' % traceback.format_exc())
# #             print('测试点4')
        end = time.time()
        seconds = end - start
        if seconds:
            fps1 = int(1 / seconds)
#             cv2.putText(frame, str(fps1), (33,33), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        cv2.imshow('...', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
capture.release()
cv2.destroyAllWindows()

In [ ]:
# 模板匹配展示
import cv2
import numpy as np
from itertools import combinations,permutations
from imutils.perspective import four_point_transform
import math
import time
from PIL import Image, ImageDraw, ImageFont

kpcishu = 7
wkpcishu = 15
def rotateRect(img,points):
    color=(0,255,0)
    for i in range(4):
        cv2.line(img,tuple(points[i]),tuple(points[(i+1)%4]),color)
        
def add_chinese(img,name,text_position):

    img_PIL = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
 
    font = ImageFont.truetype('simhei.ttf', 30, encoding="utf-8")
    
    draw = ImageDraw.Draw(img_PIL)
    draw.text(text_position, '识别结果：' + name, font=font, fill=(255,0,0))

    img_OpenCV = cv2.cvtColor(np.asarray(img_PIL),cv2.COLOR_RGB2BGR)

    return img_OpenCV

def f(img):
    row,col,c=img.shape
    img_gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    ret,img_thre=cv2.threshold(img_gray,127,255,cv2.THRESH_BINARY)

    kernel=np.ones([5,5])
    img_erode = cv2.erode(img_thre, kernel)
    img_dilate=cv2.dilate(img_erode,kernel)
#     cv2.imshow('open', img_dilate)
    contours,hierarchy = cv2.findContours(img_dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    count=0
    images=[]
    points=[]
    for cnt in contours:
        # 轮廓过滤
        x, y, w, h = cv2.boundingRect(cnt)
        if x<10 or y<10  or (x+w)>(col-10) or (y+h)>(row-10):continue#边界过滤
        if abs(w/(h*1.0)-1)>0.3:
#             print(abs(w/(h*1.0)-1))
#             print('正方形过滤')
            continue#正方形过滤
        sclae=w*h/(row*col)
        if sclae<0.03 or sclae>0.8:
#             print(sclae)
#             print('比例过滤')
            continue#比例过滤

        if (w <= 50 and h <= 50):
#             print('大小过滤')
            continue#大小过滤
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
#         rotateRect(img, box)
        # print(box)
        warped = four_point_transform(img, box)#获取卡片

        temp=img[y:y+h,x:x+w,:]
        images.append(warped)
        points.append(box)
        count+=1
    if len(images)<=0:
        return None
    else:
        return images, points

def template_demo(tpl):
    target = cv2.imread("temp/final.png")
    th, tw = tpl.shape[:2]
    result = cv2.matchTemplate(target, tpl, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    tl=max_loc
    br = (tl[0] + tw, tl[1] + th)  # br是矩形右下角的点的坐标
    point=(int((tl[0]+br[0])/2),int((tl[1]+br[1])/2))#计算矩形中心
    cv2.rectangle(target, tl, br, (0, 0, 255), 2)
    cv2.circle(target,point,3,(0,255,0))
    # print(point)
    index=int(point[1]/100)*7+int(point[0]/100)
    # print(index)
    # cv2.namedWindow("match")
    cv2.imshow("match", target)
    if abs(point[0]%100-50)>10 or abs(point[1]%100-50)>10:
#         print('非中心点')
        return -3
#     print(result[tl])
    if result[tl]<0.2:
#         print('相似度低')
        return -3
#     print(index)
    return index

def getRe(frame, no_num, cur_flag, chuxian_num, kapian_cur, kapian_arr, zhanshi):
    cards, points = f(frame)
    if type(cards) != type(None):
        for i in range(len(cards)):
            card = cards[i]
            box = points[i]
            card = cv2.resize(card, (100, 100))
    #         card = card[10:90,10:90,:]
#             cv2.imshow('card', card)
            index = template_demo(card)
            res = words[index]
            # print(res)
            if index in range(4,14):
                zhanshi = 1
                if no_num <= 0:
                    cur_flag = 1
                    chuxian_num = kpcishu
                    no_num = wkpcishu
                    kapian_cur = 'kapian'
                if chuxian_num == 0 and cur_flag:
                    kapian_arr.append(kapian_cur)
                    print(kapian_arr)
                    cur_flag = 0
                if chuxian_num in range(1,kpcishu-2):
                    if res == kapian_cur:
                        chuxian_num -= 1
                if chuxian_num in [kpcishu-2, kpcishu-1]:
                    if res == kapian_cur:
                        chuxian_num -= 1
                    else:
                        chuxian_num = kpcishu
    #             print('ceshiu')
                if chuxian_num == kpcishu:
    #                 print('10:{}'.format(res))
                    kapian_cur = res
                    chuxian_num -= 1

                    
            if res:
                frame = add_chinese(frame,res,(0, 60))
                rotateRect(frame, box)
#             print(index)
            if res == '掉头' and zhanshi:
                zhanshi = 0
                print('输出字符串结果：' + ''.join(kapian_arr))
                kapian_arr = []
    else:
        return None
    return frame, no_num, cur_flag, chuxian_num, kapian_cur, kapian_arr, zhanshi,card
    
    
# words=['(', ')', '+', '-', '=', '/', '*', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C',
#           'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
#           'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '掉头', '前进', '停车', '右转', '左转', 0]
# words=['(', ')', '+', '-', '=', '/', '*', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C',
#           'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
#           'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'backward', 'forward', 'stop', 'right', 'left', 0]
words=['(', ')', '+', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'A', 'B', 'C',
          'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
          'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '*', '前进', '停车', '右转', '左转', 0, '掉头', '/']

cur_flag = 1
kapian_arr = []
chuxian_num = kpcishu
no_num = wkpcishu
zhanshi = 1
kapian_cur = 'kapian'
ii = 0

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    try:
#         start = time.time()
        frame_t = frame.copy()
        frame_t, no_num, cur_flag, chuxian_num, kapian_cur, kapian_arr, zhanshi,ccc = \
                getRe(frame_t, no_num, cur_flag, chuxian_num, kapian_cur, kapian_arr, zhanshi)
#         print('123123')
#         end = time.time()
#         seconds = end - start
#         if seconds:
#             fps1 = int(1 / seconds)
#             cv2.putText(frame, str(fps1), (33,33), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        cv2.imshow('out', frame_t)
    except Exception as e:
#         print(e)
        no_num -= 1
        try:
            cv2.imshow('out', frame)
        except:
            pass
#         pass
    if cv2.waitKey(4) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows() 

In [2]:
# 候选框过滤展示
import cv2
import numpy as np
from itertools import combinations,permutations
from imutils.perspective import four_point_transform
import math
import time
from PIL import Image, ImageDraw, ImageFont

kpcishu = 7
wkpcishu = 15
def rotateRect(img,points):
    color=(0,255,0)
    for i in range(4):
        cv2.line(img,tuple(points[i]),tuple(points[(i+1)%4]),color)
        
def add_chinese(img,name,text_position):

    img_PIL = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
 
    font = ImageFont.truetype('simhei.ttf', 30, encoding="utf-8")
    
    draw = ImageDraw.Draw(img_PIL)
    draw.text(text_position, '识别结果：' + name, font=font, fill=(255,0,0))

    img_OpenCV = cv2.cvtColor(np.asarray(img_PIL),cv2.COLOR_RGB2BGR)

    return img_OpenCV

def f(img):
    row,col,c=img.shape
    img_gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    ret,img_thre=cv2.threshold(img_gray,127,255,cv2.THRESH_BINARY)

    kernel=np.ones([5,5])
    img_erode = cv2.erode(img_thre, kernel)
    img_dilate=cv2.dilate(img_erode,kernel)
#     cv2.imshow('open', img_dilate)
    contours,hierarchy = cv2.findContours(img_dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    count=0
    images=[]
    points=[]
    for cnt in contours:
        # 轮廓过滤
        x, y, w, h = cv2.boundingRect(cnt)
        if x<10 or y<10  or (x+w)>(col-10) or (y+h)>(row-10):continue#边界过滤
#         if abs(w/(h*1.0)-1)>0.3: continue#正方形过滤
#         sclae=w*h/(row*col)
#         if sclae<0.03 or sclae>0.8: continue#面积占比过滤

        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
#         rotateRect(img, box)
        # print(box)
        warped = four_point_transform(img, box)#获取卡片

        temp=img[y:y+h,x:x+w,:]
        images.append(warped)
        points.append(box)
        count+=1
    if len(images)<=0:
        return None
    else:
        return images, points

def template_demo(tpl):
    target = cv2.imread("temp/final.png")
    th, tw = tpl.shape[:2]
    result = cv2.matchTemplate(target, tpl, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    tl=max_loc
    br = (tl[0] + tw, tl[1] + th)  # br是矩形右下角的点的坐标
    point=(int((tl[0]+br[0])/2),int((tl[1]+br[1])/2))#计算矩形中心
    cv2.rectangle(target, tl, br, (0, 0, 255), 2)
    cv2.circle(target,point,3,(0,255,0))
    # print(point)
    index=int(point[1]/100)*7+int(point[0]/100)
    # print(index)
    # cv2.namedWindow("match")
    cv2.imshow("match", target)
    if abs(point[0]%100-50)>10 or abs(point[1]%100-50)>10:
#         print('非中心点')
        return -3
#     print(result[tl])
    if result[tl]<0.2:
#         print('相似度低')
        return -3
#     print(index)
    return index

def getRe(frame, no_num, cur_flag, chuxian_num, kapian_cur, kapian_arr, zhanshi):
    cards, points = f(frame)
    for i in range(len(cards)):
        card = cards[i]
        box = points[i]
        rotateRect(frame, box)
    cv2.imshow('houxuan', frame)
    return None
    
    
# words=['(', ')', '+', '-', '=', '/', '*', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C',
#           'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
#           'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '掉头', '前进', '停车', '右转', '左转', 0]
# words=['(', ')', '+', '-', '=', '/', '*', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C',
#           'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
#           'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'backward', 'forward', 'stop', 'right', 'left', 0]
words=['(', ')', '+', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'A', 'B', 'C',
          'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
          'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '*', '前进', '停车', '右转', '左转', 0, '掉头', '/']

cur_flag = 1
kapian_arr = []
chuxian_num = kpcishu
no_num = wkpcishu
zhanshi = 1
kapian_cur = 'kapian'
ii = 0

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    try:
#         start = time.time()
        frame_t = frame.copy()
        _ = getRe(frame_t, no_num, cur_flag, chuxian_num, kapian_cur, kapian_arr, zhanshi)
        cv2.imshow('out', frame_t)
    except Exception as e:
#         print(e)
        no_num -= 1
        try:
            cv2.imshow('out', frame)
        except:
            pass
#         pass
    if cv2.waitKey(4) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows() 